# Interactive PDQ hasher

This "Python notebook" lets you interactively edit images, and automatically compute SHA2-256 and PDQ hashes and their Hamming distance.

## How to run

To get this code running, in the menu bar immediately below the "jupyter" logo, click `Run > Run All Cells`.

Once this is done, you can scroll to the end of the notebook, where you will find an image and a series of sliders. The result should look something like this.

![Example](example.png)

Each slider controls a change to the image. Move them one by one to apply different changes.

It can take a couple of seconds for the change to take place and the hashes to be recomputed.

To try this on a different image, follow the instructions under "Choose your picture" that you can find towards the bottom of this notebook file.



In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider, Text
import ipywidgets as widgets
from PIL import Image, ImageFilter, ImageOps

In [2]:
def hamming_distance(s1, s2):
    """
        Computes the hamming distance between two hexstrings.
    """
    b1 = bytes.fromhex(s1)
    b2 = bytes.fromhex(s2)
    i1 = int.from_bytes(b1, byteorder='little')
    i2 = int.from_bytes(b2, byteorder='little')
    hd = (i1 ^ i2).bit_count()
    return hd

In [ ]:
import pdqhash, numpy

def pdq_from_image(img):
    pil_image = img.convert('RGB')
    open_cv_image = numpy.array(pil_image)
    hash_vector, quality = pdqhash.compute(open_cv_image)
    digest = b""
    for i in range(len(hash_vector)//8):
        bits = hash_vector[8*i:8*(i+1)]
        b = int(sum(bits[i] * 2**(7-i) for i in range(8)))
        digest += b.to_bytes(1, byteorder='little')
    return digest.hex()


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import hashlib
from io import BytesIO

def sha256_from_image(img):
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    m = hashlib.sha256()
    m.update(img_byte_arr)
    return m.hexdigest()

In [5]:
# from io import BytesIO
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import colorsys

def img2array(im):
    if im.mode != 'RGB':
        im = im.convert(mode='RGB')
    return np.fromstring(im.tostring(), dtype='uint8').reshape((im.size[1], im.size[0], 3))

def hue_shift(img, hue_shift=0):
    # Convert image to RGB if it's not already
    img = img.convert('RGB')

    # Get image data as a list of RGB tuples
    data = img.getdata()

    # Convert RGB to HSV, shift the hue, and convert back to RGB
    new_data = []
    for item in data:
        r, g, b = item
        h, s, v = colorsys.rgb_to_hsv(r/255., g/255., b/255.)
        h = (h + hue_shift/360) % 1
        r, g, b = colorsys.hsv_to_rgb(h, s, v)
        new_data.append((int(r*255), int(g*255), int(b*255)))

    # Create a new image with the shifted colors
    new_img = Image.new(img.mode, img.size)
    new_img.putdata(new_data)

    return new_img


In [6]:
import requests
import functools

@functools.cache
def get_img_from_url(url):
    response = requests.get(url, headers={'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:61.0) Gecko/20100101 Firefox/61.0" })
    img = Image.open(BytesIO(response.content))
    return img

In [10]:
def edit_image(
    url,
    crop_left=0,
    crop_top=0,
    crop_right=0,
    crop_bottom=0,
    hue=0,
    rotate=0,
    scale=1,
    blur=0,
    pixelate=1,
    posterize=8,
    solarize=255
    ):

    if url[:4] == "http":
        img = get_img_from_url(url)
    else:
        with Image.open(url) as img:
            img.load()

    og_pdq_digest = str(pdq_from_image(img))
    og_sha256_digest = str(sha256_from_image(img))
    width, height = img.size
    edited = img

    # crop
    edited = edited.crop(tuple(map(round, [
        width * crop_left, height * crop_top, width * (1-crop_right), height * (1-crop_bottom)
    ])))

    # hue shift
    if hue % 360 != 0:
        edited = hue_shift(edited, hue)

    # scale
    edited = edited.resize((round(edited.width * scale), round(edited.height * scale)))

    # rotate
    edited = edited.rotate(rotate)

    # gaussian blur
    edited = edited.filter(ImageFilter.GaussianBlur(blur))

    # pixelate
    edited = edited.filter(ImageFilter.ModeFilter(pixelate))

    # posterize
    edited = ImageOps.posterize(edited, posterize)

    # solarize
    edited = ImageOps.solarize(edited, solarize)

    # report back
    pdq_digest = str(pdq_from_image(edited))
    sha256_digest = str(sha256_from_image(edited))

    print(
        f"Shown:",
        f"crop_left={crop_left}",
        f"crop_top={crop_top}",
        f"crop_right={crop_right}",
        f"crop_bottom={crop_bottom}",
        f"hue={hue}",
        f"rotate={rotate}\n",
        f"      scale={scale}",
        f"blur={blur}",
        f"pixelate={pixelate}",
        f"posterize={posterize}",
        f"solarize={solarize}\n"
    )

    print("Original SHA256:", og_sha256_digest, "(as PNG file)")
    print("Modified SHA256:", sha256_digest, "(as PNG file) --- Hamming distance:", hamming_distance(og_sha256_digest, sha256_digest))
    print(   "Original PDQ:   ", og_pdq_digest)
    print("Modified PQD:   ", pdq_digest, " --- Hamming distance:", hamming_distance(og_pdq_digest, pdq_digest))
    return edited

# Choose your picture

In the following code block, you can choose what image to analyse.
Simply set `picture = "..."`, and replace `...` with either one of the following values, or with a url pointing to an image online.

### Example pictures
- `images/am-football.jpg`
- `images/bavaria.jpg`
- `https://www.surrey.ac.uk/sites/default/files/styles/640x360_16_9/public/2019-08/cyber-security-fingerprint-computer-science-highlight.webp?itok=VTaB2Dft`

In [14]:
picture = "images/am-football.jpg"

In [15]:
interact(
    edit_image,
    url=Text(value=picture, placeholder='image url or path', disabled=False),
    crop_left=FloatSlider(min=0., max=1., step=.1, value=0.),
    crop_top=FloatSlider(min=0., max=1., step=.1, value=0.),
    crop_right=FloatSlider(min=0., max=1., step=.1, value=0.),
    crop_bottom=FloatSlider(min=0., max=1., step=.1, value=0.),
    hue=IntSlider(min=0, max=360, step=1, value=0),
    scale=FloatSlider(min=0., max=2., step=.1, value=1.),
    rotate=IntSlider(min=-180, max=180, step=1, value=0),
    blur=IntSlider(min=0, max=100, step=1, value=0),
    pixelate=IntSlider(min=1, max=100, step=1, value=1),
    posterize=IntSlider(min=1, max=8, step=1, value=8),
    solarize=IntSlider(min=0, max=256, step=1, value=256)
);

interactive(children=(Text(value='images/am-football.jpg', description='url', placeholder='image url or path')…